## Download Library Sastrawi

In [2]:
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

## Import Library

In [14]:
# Import Library
import pandas as pd
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk

#Download NLTK Stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Preprocessing Text

In [6]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

stop_words = set(stopwords.words('indonesian'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    tokens = text.split()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

## Load Dataset

In [4]:
df = pd.read_csv("dataset_ulasan_shopee.csv")

## Labeling

In [7]:
# Labeling berdasarkan kata kunci
positive_keywords = ["bagus", "mantap", "cepat", "puas", "keren", "terbaik", "menyenangkan", "lancar", "responsif", "oke"]
negative_keywords = ["jelek", "lemot", "buruk", "error", "gagal", "parah", "kecewa", "bug", "force close"]

def label_sentiment(text):
    text = text.lower()
    if any(word in text for word in positive_keywords):
        return "positif"
    elif any(word in text for word in negative_keywords):
        return "negatif"
    else:
        return "netral"

if "label" not in df.columns:
    df["label"] = df["review"].apply(label_sentiment)

# Filter dan preprocessing
df = df[df['label'].isin(['positif', 'netral', 'negatif'])]
df["clean_review"] = df["review"].apply(preprocess_text)

## Split Dataset

In [8]:
X = df["clean_review"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Traning

In [9]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

### Model 1: Logistic Regression

In [10]:
# Logisitic Regression
lr = LogisticRegression(max_iter=300)
lr.fit(X_train_tfidf, y_train)
y_pred_train_lr = lr.predict(X_train_tfidf)
y_pred_lr = lr.predict(X_test_tfidf)
acc_train_lr = accuracy_score(y_train, y_pred_train_lr) * 100
acc_test_lr = accuracy_score(y_test, y_pred_lr) * 100

### Model 2: SVM

In [11]:
# SVM
svm = SVC()
svm.fit(X_train_tfidf, y_train)
y_pred_train_svm = svm.predict(X_train_tfidf)
y_pred_svm = svm.predict(X_test_tfidf)
acc_train_svm = accuracy_score(y_train, y_pred_train_svm) * 100
acc_test_svm = accuracy_score(y_test, y_pred_svm) * 100

### Model 3: Random Forest

In [12]:
# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train_tfidf, y_train)
y_pred_train_rf = rf.predict(X_train_tfidf)
y_pred_rf = rf.predict(X_test_tfidf)
acc_train_rf = accuracy_score(y_train, y_pred_train_rf) * 100
acc_test_rf = accuracy_score(y_test, y_pred_rf) * 100

## Evaluasi Tiap Model

In [13]:
print("\nAkurasi Logistic Regression - Training: {:.2f}%".format(acc_train_lr))
print("Akurasi Logistic Regression - Testing : {:.2f}%".format(acc_test_lr))

print("\nAkurasi SVM - Training: {:.2f}%".format(acc_train_svm))
print("Akurasi SVM - Testing : {:.2f}%".format(acc_test_svm))

print("\nAkurasi Random Forest - Training: {:.2f}%".format(acc_train_rf))
print("Akurasi Random Forest - Testing : {:.2f}%".format(acc_test_rf))

print("\nClassification Report - Logistic Regression:")
print(classification_report(y_test, y_pred_lr))


Akurasi Logistic Regression - Training: 100.00%
Akurasi Logistic Regression - Testing : 100.00%

Akurasi SVM - Training: 100.00%
Akurasi SVM - Testing : 100.00%

Akurasi Random Forest - Training: 100.00%
Akurasi Random Forest - Testing : 100.00%

Classification Report - Logistic Regression:
              precision    recall  f1-score   support

     negatif       1.00      1.00      1.00        59
      netral       1.00      1.00      1.00       213
     positif       1.00      1.00      1.00       378

    accuracy                           1.00       650
   macro avg       1.00      1.00      1.00       650
weighted avg       1.00      1.00      1.00       650

